In [ ]:
pip install nba_api

In [ ]:
from nba_api.stats.static import players
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))

In [ ]:
year = "2019-20" #Can change this to any specific season with format example: 2017-18
name = ""
fantasyStats = []
count = 0
from nba_api.stats.endpoints import playercareerstats
playerIDs = []
#Retieve every single NBA player ID for all past and present NBA players
for player in nba_players:
    playerIDs.append(str(player.get("id")))
len(playerIDs)

In [ ]:
headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
} #Headers for the player stats api call

In [ ]:
#Compiles the player stats for a specific season for each player
def stats(player,count):
    career = playercareerstats.PlayerCareerStats(player_id=playerIDs[player])
    careerdict = career.get_dict()
    fantasyStatsTemp = []
    Team = ""
    GP = 0
    PPG = 0
    APG = 0
    RPG = 0
    BPG = 0
    TOPG = 0
    FGPercent = 0
    FTPercent = 0
    ThreesPG = 0
    FER = 0 
    #These are weights calculated based on previous fantasy stats data to represent the importance or weight of each stat in fasntasy basketball
    APGWeight = 5.02
    RPGWeight = 2.7
    SPGWeight = 17.41
    BPGWeight = 19.19
    ThreesPGWeight = 10.27
    FGPercentWeight = 30.271
    FTPercentWeight = 48.803
    TOPGWeight = -10.44
    
    for key in careerdict:
        if key == "resultSets":
            item = careerdict["resultSets"]
        
    for key2 in item[0]:
        if key2 == "rowSet":
            newItem = item[0]
            seasons = newItem["rowSet"]
            for season in seasons:
                if season[1] == year:
                    for player in nba_players:
                        if player.get("id") == season[0]:
                            name = player.get("full_name")
                    #Retrieve relevant player data and append into a temporary list which is then appended to a 2D list with other players
                    Team = season[4]
                    GP = season[6]
                    PPG = season[26]/GP
                    APG = season[21]/GP
                    RPG = season[20]/GP
                    SPG = season[22]/GP
                    BPG = season[23]/GP
                    TOPG = season[24]/GP
                    FGPercent = season[11]
                    FTPercent = season[17]
                    ThreesPG = season[12]/GP
                    FER = PPG + APG * APGWeight + RPG * RPGWeight + SPG * SPGWeight + BPG * BPGWeight + ThreesPG * ThreesPGWeight + FGPercent * FGPercentWeight + FTPercent * FTPercentWeight + TOPG * TOPGWeight
                    fantasyStatsTemp.append(name)
                    fantasyStatsTemp.append(Team)
                    fantasyStatsTemp.append(GP)
                    fantasyStatsTemp.append(PPG)
                    fantasyStatsTemp.append(APG)
                    fantasyStatsTemp.append(RPG)
                    fantasyStatsTemp.append(SPG)
                    fantasyStatsTemp.append(BPG)
                    fantasyStatsTemp.append(TOPG)
                    fantasyStatsTemp.append(FGPercent)
                    fantasyStatsTemp.append(FTPercent)
                    fantasyStatsTemp.append(ThreesPG)
                    fantasyStatsTemp.append(FER)
                    fantasyStats.append(fantasyStatsTemp)

In [ ]:
for count in range(len(playerIDs)):
    stats(count,count)
    playerIDs.remove(playerIDs[count])
    print(len(playerIDs))

In [ ]:
len(fantasyStats)

In [ ]:
len(playerIDs) #When this list is empty that means all players from that season were added to the fantasyStats list

In [ ]:
count = -1
size = len(fantasyStats)-1

#Loop through the fantasyStats list and check for players with multiple entries per season due to trade
for player in fantasyStats:
    count += 1
    if player[-12] == "TOT":
        Totals = []
        Totals.append(player[-13])
        Totals.append(player[-25])
        Totals.append(player[-11])
        Totals.append(player[-10])
        Totals.append(player[-9])
        Totals.append(player[-8])
        Totals.append(player[-7])
        Totals.append(player[-6])
        Totals.append(player[-5])
        Totals.append(player[-4])
        Totals.append(player[-3])
        Totals.append(player[-2])
        Totals.append(player[-1])
        fantasyStats.remove(fantasyStats[count])
        fantasyStats.append(Totals)#Append the players totals with his last team of the season and remove the previous entry

count = 0
for count in range(size-1):
    if fantasyStats[count][0] == fantasyStats[count+1][0]:
        fantasyStats.remove(fantasyStats[count])
        count = count - 1
        
len(fantasyStats)

In [ ]:
def split(arr,low,high): 
    i = (low-1)         #Index of smaller element 
    splitPoint = arr[high][-1]     
  
    for j in range(low,high): 
        if   arr[j][-1] <= splitPoint: 
            # increment index of smaller element 
            i = i+1 
            arr[i],arr[j] = arr[j],arr[i] 
  
    arr[i+1],arr[high] = arr[high],arr[i+1] 
    return (i+1) 

def quickSort(arr,low,high): 
    if low < high: 
  
        splitIndex = split(arr,low,high) 
        quickSort(arr, low, splitIndex-1) 
        quickSort(arr, splitIndex, high) 

In [ ]:
quickSort(fantasyStats,0,len(fantasyStats)-1)
fantasyStats.reverse()
f = open("Stats.txt", "w")
f.write(str(fantasyStats))
f.close()
fantasyStats

In [ ]:
pip install --upgrade firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
if not firebase_admin._apps:
    cred = credentials.Certificate("nbafantasystats-c91bc-firebase-adminsdk-fumaf-8ba51a7433.json")
    firebase_admin.initialize_app(cred)
    db = firestore.client()

In [ ]:
for player in fantasyStats:
    doc_ref = db.collection(year).document(player[0].upper())
    doc_ref.set({
        u'Name': player[0],
        u'Team': player[1],
        u'GP': player[2],
        u'PPG': player[3],
        u'APG': player[4],
        u'RPG': player[5],
        u'SPG': player[6],
        u'BPG': player[7],
        u'TOPG': player[8],
        u'FGPercent': player[9],
        u'FTPercent': player[10],
        u'ThreesPG': player[11],
        u'FER': player[12],
    })